In [1]:
import requests
import bs4
from bs4 import BeautifulSoup as soup
import datetime
from pytz import all_timezones
import pytz
from pymongo import MongoClient

In [7]:
client = MongoClient('mongodb://localhost:27017/')
db = client['crime_app']
table=db['crimes']

In [4]:
editable_list=db['list_of_resources']
editable_list=editable_list.find({'Network':'SpotCrime'})
list_of_cities=[]
for element in editable_list:
    list_of_cities.append(element['Location'])
list_of_cities    


['new york , new york', 'Los Angeles, California', 'San Francisco, California']

In [5]:
if len(list_of_cities) == 0:
    print('Could Not fetch list of cities from database')

In [6]:
datetime_now=datetime.datetime.now(tz=pytz.timezone('EET'))
print(datetime_now)
delta=datetime.timedelta(days=10)
datetime_now=datetime_now-delta
print(datetime_now)
cursor=table.find({'Network':'SpotCrime','Location':{'$in':list_of_cities},'Date':{'$gte':datetime.datetime.strptime(datetime_now.strftime('%Y-%m-%d %H:%M:%S'),'%Y-%m-%d %H:%M:%S')}})

2020-09-26 10:10:08.685718+03:00
2020-09-16 10:10:08.685718+03:00


In [7]:
old_list=[]
for row in cursor:
    
    old_list.append(row['Key'])   

In [23]:
old_list

['https://spotcrime.com/crime/146870959-a0e8048db56f8657a02ecca964d28065',
 'https://spotcrime.com/crime/146870960-ab60f7c880787c401dcb2263ef5e5dd2',
 'https://spotcrime.com/crime/146900337-a060100be1df9cf657d1afa47e3db9b5',
 'https://spotcrime.com/crime/146890828-2c61fb09f6b14982e44d0b1fb7cca235',
 'https://spotcrime.com/crime/146890825-060a43df8e3d3c8f7afc87fbea1eb0ac',
 'https://spotcrime.com/crime/146890821-50bf1c51cf0a68bace7da1398975e9b9',
 'https://spotcrime.com/crime/146890818-633777aa5dcdd5ab49e2a8bfd4776928',
 'https://spotcrime.com/crime/146890814-fb1792bcb104bf2834c132fd78d034bc',
 'https://spotcrime.com/crime/146890811-14e7e06d2ac6bf340d91a6b79f37bb4b',
 'https://spotcrime.com/crime/146890804-a466783e2b575bf1c3b353daf1b4ae41',
 'https://spotcrime.com/crime/146890808-7e2d116e1b4ed96084099b14a352e4cc',
 'https://spotcrime.com/crime/146890798-ff51b4175887130a778a1add06085074',
 'https://spotcrime.com/crime/146890801-48a3c34befd96bfe7400410e312a01ca',
 'https://spotcrime.com/c

In [8]:
list_of_links=[]
list_of_types=[]
list_of_datetime=[]
list_of_description=[]
list_of_address=[]
list_of_map_link=[]
list_of_latitude=[]
list_of_longitude=[]
cit=[]
website=[]

In [10]:
j=1

data=requests.get('https://spotcrime.com/browse-by-state')
data=data.content
page=soup(data,'lxml')
states_list=page.find_all('div', class_='state-page__state-container')

for city in list_of_cities:
    try:
        state=city.split(',')[1].strip().lower()
        for the_state in states_list:
            if the_state.find('a').text.strip().lower() == state.lower():
                state_link=the_state.find('a')['href']
                state_link='https://spotcrime.com'+state_link
                print(state_link)
            
    
        city=city.split(',')[0].strip().lower()
        inside_state=requests.get(state_link)
        inside_state=inside_state.content
        page_city=soup(inside_state,'lxml')
        found_cities=page_city.find('div' ,id='state-page__main-areas')
        found_cities=found_cities.find_all('a')
        for the_find in found_cities:
            if the_find.text.split(',')[0].strip().lower() == city.lower():
                city_link=the_find['href']
                city_link='https://spotcrime.com'+city_link
                print(city_link)
            
        
        crime_data=requests.get(city_link)
        crime_data=crime_data.content
        page_crime=soup(crime_data,'lxml')
        crimes=page_crime.find_all('a', class_='city-page__crime-card')
        for links in crimes:
            url=links['href']
            if url in old_list:
                raise Exception
            list_of_links.append(url)
            crimedata=requests.get(url)
            crimedata=crimedata.content
            page=soup(crimedata,'lxml')
            description=page.find('div',class_='crime-details-page__details-section')
            description=description.find_all('p')
            types=description[0].text
            types=types.split(':')[1].strip()
            
            list_of_types.append(types)
            date_time=description[1].text
    
            date_time=date_time.split(':')
        
            date_time=date_time[1:]
            
            date_time=date_time[0].strip()+':'+date_time[1].strip()
            
            date_time_obj = datetime.datetime.strptime(date_time, '%m/%d/%Y %I:%M %p')
            tz = pytz.timezone('US/Central')
            date_time_obj = date_time_obj.replace(tzinfo=tz)
            date_time_obj=  date_time_obj.astimezone(pytz.timezone('EET'))
        
            list_of_datetime.append(date_time_obj)
            desc_of_crime=description[2].text
            desc_of_crime=desc_of_crime.split(':')[1].strip()
            #print(desc_of_crime)
            list_of_description.append(desc_of_crime)
            address=description[3].text
            address=address.split(':')[1].strip()
            #print(address)
            list_of_address.append(address)
            map_of_crime=page.find('a' , class_='crime-details-page__google-map-btn')
            map_of_crime=map_of_crime['href']
            #print(map_of_crime)
            longitude=map_of_crime.split(' ')[1]
            #print(longitude)
            longitude=float(longitude)
            list_of_longitude.append(longitude)
            latitude=map_of_crime.split('=')[2]
            latitude=latitude.split(',')
            latitude=latitude[0]
            #print(latitude)
            latitude=float(latitude)
            list_of_latitude.append(latitude)
            list_of_map_link.append(map_of_crime)
            cit.append(city)
            website.append('spotcrime')
            document={'Key':url,'Longitude':longitude, 'Latitude':latitude, 'Location':city,'Text':desc_of_crime,'Category':types, 'Address':address
                            ,  'Date':date_time_obj, 'Network': 'SpotCrime'}
            print(j)
            j=j+1
            table.insert_one(document)
                
    except:
        print('Found in old')
        continue

https://spotcrime.com/NY
https://spotcrime.com/NY/New%20York
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
https://spotcrime.com/CA
https://spotcrime.com/CA/Los%20Angeles
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
https://spotcrime.com/CA
https://spotcrime.com/CA/San%20Francisco
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150


In [26]:
import pandas as pd

In [27]:
df=pd.DataFrame({'Key': list_of_links,'Type':list_of_types,'Description':list_of_description,'Address':list_of_address,'DateTime':list_of_datetime,'Latitude':list_of_latitude,'Longitude':list_of_longitude, 'City':cit,'Website':website})

In [28]:
df.head()

,Key,Type,Description,Address,DateTime,Latitude,Longitude,City,Website


In [38]:
df.to_csv('SpotCrime.csv', index=False)

In [3]:
for time in all_timezones:
    if 'US' in time:
        print(time)

US/Alaska
US/Aleutian
US/Arizona
US/Central
US/East-Indiana
US/Eastern
US/Hawaii
US/Indiana-Starke
US/Michigan
US/Mountain
US/Pacific
US/Samoa


In [5]:
print(datetime.datetime.now(tz=pytz.timezone('US/Pacific')))

2020-09-29 00:58:25.583531-07:00


In [2]:
for time in all_timezones:
    #if 'US' in time:
        #db['timezones'].insert_one({'timezone':time})
    print(time)    

Africa/Abidjan
Africa/Accra
Africa/Addis_Ababa
Africa/Algiers
Africa/Asmara
Africa/Asmera
Africa/Bamako
Africa/Bangui
Africa/Banjul
Africa/Bissau
Africa/Blantyre
Africa/Brazzaville
Africa/Bujumbura
Africa/Cairo
Africa/Casablanca
Africa/Ceuta
Africa/Conakry
Africa/Dakar
Africa/Dar_es_Salaam
Africa/Djibouti
Africa/Douala
Africa/El_Aaiun
Africa/Freetown
Africa/Gaborone
Africa/Harare
Africa/Johannesburg
Africa/Juba
Africa/Kampala
Africa/Khartoum
Africa/Kigali
Africa/Kinshasa
Africa/Lagos
Africa/Libreville
Africa/Lome
Africa/Luanda
Africa/Lubumbashi
Africa/Lusaka
Africa/Malabo
Africa/Maputo
Africa/Maseru
Africa/Mbabane
Africa/Mogadishu
Africa/Monrovia
Africa/Nairobi
Africa/Ndjamena
Africa/Niamey
Africa/Nouakchott
Africa/Ouagadougou
Africa/Porto-Novo
Africa/Sao_Tome
Africa/Timbuktu
Africa/Tripoli
Africa/Tunis
Africa/Windhoek
America/Adak
America/Anchorage
America/Anguilla
America/Antigua
America/Araguaina
America/Argentina/Buenos_Aires
America/Argentina/Catamarca
America/Argentina/ComodRivad

In [2]:
st='2020-09-10 15:00'

In [3]:
dt1=datetime.datetime.strptime(st, '%Y-%m-%d %H:%M')

In [4]:
dt1

datetime.datetime(2020, 9, 10, 15, 0)

In [5]:
tz = pytz.timezone('US/Pacific')
dt1=tz.localize(dt1)

In [6]:
dt1.strftime('%Y-%m-%d %H:%M')

'2020-09-10 15:00'

In [7]:
dt2=dt1.astimezone(pytz.timezone('EET'))

In [8]:
dt2.strftime('%Y-%m-%d %H:%M')

'2020-09-11 01:00'